http://desktop.arcgis.com/en/arcmap/10.3/tools/spatial-analyst-toolbox/how-slope-works.htm

In [4]:
# raster slope calculation
import numpy as np
ta = np.array([[50.,45.,50.],[30.,30.,30.],[8.,10.,10.]])
print(np.shape(ta))
print(ta)

(3, 3)
[[50 45 50]
 [30 30 30]
 [ 8 10 10]]


In [13]:
i=1
j=1
dx = 5;
dy = 5;
dzdx=((ta[i-1,j+1]+2.*ta[i,j+1]+ta[i+1,j+1])-(ta[i-1,j-1]+2.*ta[i,j-1]+ta[i+1,j-1]))/(8.*dx)
dzdy=((ta[i+1,j-1]+2.*ta[i+1,j]+ta[i+1,j+1])-(ta[i-1,j-1]+2.*ta[i-1,j]+ta[i-1,j+1]))/(8.*dy)
riseorun = (dzdx**2.+dzdy**2.)**(0.5)
print(dzdx, dzdy)
print(riseorun)

0.05 -3.8
3.80032893313
